In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
from lightgbm import LGBMClassifier
from sklearn.metrics import matthews_corrcoef as matt
from hyperopt import hp
from hyperopt import tpe
from hyperopt import Trials
from hyperopt import fmin

In [ ]:
"""
Reading the data. Data has stock tickers in the first row and dates in the first column. Only trading days are used.
"""
df = pd.read_excel('C:/Users/FS_Askar_A/midcap.xlsx', index_col="Date")

In [ ]:
def create_returns(dtf, tick, lead=5, lags=[1,2,3,4,5,7,10,15,20,30,50], tr=0.025):
    """
    Calculates forward and lagged returns for a stock. Also creates target column
    for returns over some threshold.
    
    Parameters:
    -----------
    dtf - data with all stock returns
    tick - ticker of stock 
    lead - forward return lead
    lags - lagged returns
    tr - threshold value for target variable
    
    Returns:
    --------
    DataFrame for one stock with its price, forward and lagged returns and target column.
    """
    dtf = dtf[[tick]]
    dtf['fwd'] = dtf[tick].shift(-lead) / dtf[tick]-1
    for lag in lags:
        name = 'ret'+str(lag)
        dtf[name] = dtf[tick]/dtf[tick].shift(lag)-1
    dtf['ycol'] = np.where(dtf['fwd'] >= tr, 1, 0)
    return dtf.dropna()


def my_tss(dtf, split, train_size=1000, test_size=100):
    """
    My take on time series split. It separates dataframe into "past" and 
    "future" from a splitting point. Also removes extra columns. 
    
    Parameters:
    dtf - dataframe of a stock
    split - splitting point in trading days
    train_size=1000 - size of the training sample
    test_size=100 - size of the testing sample
    
    Returns:
    trainx - training features
    trainy - training target column
    testx - testing features
    testy - testing target column
    
    Attention!
    This function is not fool-proof - it doesn't check whether the size of 
    the train size is less than splitting point or if testing sample has
    any values. This is done for speed purposes as it is called thousands of times.
    """
    train_start = split - train_size
    test_end = split + test_size
    trainx = dtf.drop(columns=[dtf.columns[0], 'fwd', 'ycol']).iloc[train_start:split]
    testx = dtf.drop(columns=[dtf.columns[0], 'fwd', 'ycol']).iloc[split:test_end]
    trainy = dtf['ycol'].iloc[train_start:split]
    testy = dtf['ycol'].iloc[split:test_end]
    return trainx, testx, trainy, testy


def integerize(d):
    """
    Converts hyperparameter values into integers. This is a compensation of hyperopt's 
    problem where it feeds integer values in float form. I.e. 2.0 instead of 2.
    
    Parameters:
    d - dictionary of hyperparameters
    
    Returns:
    d - dictionary of hyperparameters with integers where required
    """
    
    int_params = [
        'train_size',
        'test_size',
        'num_leaves',
        'max_depth',
        'n_estimators',
        'min_child_samples',
        'upto',
        'ticks_to_use'
    ]
    
    for k in d:
        if k in int_params:
            d[k] = int(d[k])
            
    return d


def calc_ret(results):
    """
    Calculation of return from predicted data.
    
    Parameters:
    -----------
    results - list of dataframes. Dataframes must have 'avg' column, which is the 
                average 5-day forward return for the day.
    
    Returns:
    --------
    ===negative=== return for the whole period, across all the dataframes. 
                Return is negative because hyperopt minimizes a function.
    """
    
    bigdf = pd.concat(results)
    bigdf['avg'] = bigdf.mean(axis=1).fillna(0)+1
    
    lead = 5
    portf = 100
    subportf = [portf/lead for l in range(lead)]
    x = 0
    for day in bigdf['avg']:
        subportf[x%5] = subportf[x%5]*day
        x+=1
        
    return 1-np.sum(subportf)/portf


def check_params(params, upto, stock_num=20):
    """
    Predicts on out-of-sample and out-of-cross-validation data
    using optimized hyperparameters. 
    
    Parameters:
    -------
    params - optimized hyperparameters
    upto - maximum split point for cross-validations,
            here it is incremented by 100 to not include the
            last validation sample (100 points in size)
            
    Returns:
    --------
    dataframe with 'avg' column, which is the 
                average 5-day forward return for the day.
    """
    params = integerize(params)
    bigdf = pd.DataFrame()
    
    for tick in df.columns[:stock_num]:
        
        tempdf = tickers_dfs[tick][['fwd']]
        trainx, testx, trainy, testy = my_tss(tickers_dfs[tick], upto+100, 
                                              train_size=params['train_size'])
        
        my_model = LGBMClassifier(num_leaves=params['num_leaves'],
                                     max_depth=params['max_depth'],
                                     learning_rate=params['learning_rate'],
                                     n_estimators=params['n_estimators'],
                                     min_child_samples=params['min_child_samples'])
        
        sw = np.where(trainy==0, params['sw'], 1)
            
        my_model.fit(trainx, trainy, sample_weight=sw)
        
        testx['pred'] = my_model.predict(testx)
        tempdf['pred'] = testx['pred']
        tempdf.dropna(inplace=True)
        tempdf['predret'] = np.where(tempdf['pred']==1, tempdf['fwd'], np.nan)
        bigdf[tick] = tempdf['predret']
        
    bigdf['avg'] = bigdf.mean(axis=1).fillna(0)+1
        
    return bigdf


def check_paramsEF(tick_params, upto):
    """
    Predicts on out-of-sample and out-of-cross-validation data
    using optimized hyperparameters for every ticker separately. 
    
    Parameters:
    -------
    tick_params - dict of optimized hyperparameters
    upto - maximum split point for cross-validations,
            here it is incremented by 100 to not include the
            last validation sample (100 points in size)
            
    Returns:
    --------
    dataframe with 'avg' column, which is the 
                average 5-day forward return for the day.
    """
    
    bigdf = pd.DataFrame()
    
    for tick, params in tick_params.items():
        
        params = integerize(params)
        
        tempdf = tickers_dfs[tick][['fwd']]
        trainx, testx, trainy, testy = my_tss(tickers_dfs[tick], upto+100, 
                                              train_size=params['train_size'])
        
        my_model = LGBMClassifier(num_leaves=params['num_leaves'],
                                     max_depth=params['max_depth'],
                                     learning_rate=params['learning_rate'],
                                     n_estimators=params['n_estimators'],
                                     min_child_samples=params['min_child_samples'])
        
        sw = np.where(trainy==0, params['sw'], 1)

        my_model.fit(trainx, trainy, sample_weight=sw)
        
        testx['pred'] = my_model.predict(testx)
        tempdf['pred'] = testx['pred']
        tempdf.dropna(inplace=True)
        tempdf['predret'] = np.where(tempdf['pred']==1, tempdf['fwd'], np.nan)
        bigdf[tick] = tempdf['predret']
        
    bigdf['avg'] = bigdf.mean(axis=1).fillna(0)+1
        
    return bigdf


def get_est_series(dtf, com=0):
    """
    Calculates total return over several out-of-cross-validation samples.
    
    Parameters:
    -----------
    dtf - dataframe with 'avg' column
    com=0 - value of commissions for ==one== side of a trade in USD/share
    
    Returns:
    portf_series - series with portfolio value at a point in time
    """
    
    comdf = dtf.drop(columns=['avg'])
    for tick in dtf.columns[:-1]:
        if df[tick].iloc[1600] >= 0:
            comdf[tick] = (df[tick].shift(-5)-com) / (df[tick]+com)-1
            comdf[tick] = np.where(dtf[tick].notna(), comdf[tick], np.nan)
        else:
            comdf[tick] = np.nan
    comdf['avg'] = comdf.mean(axis=1).fillna(0)+1
    
    portf_series = []
    lead = 5
    portf = 100
    subportf = [portf/lead for l in range(lead)]
    x = 0
    for day in comdf['avg']:
        subportf[x%5] = subportf[x%5]*day
        portf_series.append(subportf.copy())
        x+=1
    
    portf_series = pd.DataFrame(portf_series, index=comdf.index)
    portf_series = portf_series.shift(5)
    portf_series = portf_series.fillna(20).sum(axis=1)
    
    return portf_series


def get_res(params):
    
    mymodel_results = []

    for tick in df.columns:

        for split in range(1000, 2200, 100):
            trainx, testx, trainy, testy = my_tss(tickers_dfs[tick], split)
            my_model = LGBMClassifier(**params)
            my_model.fit(trainx, trainy)
            predy = my_model.predict(testx)
            mymodel_results.append(matt(testy, predy))

    return np.mean(mymodel_results)


In [ ]:
%%time

tickers_dfs = {}

for tick in tqdm_notebook(df.columns):
    tickers_dfs[tick] = create_returns(df, tick)

In [ ]:
def analyze_upto_modelF(params):
    """
    Cross-validation function that takes does hyperparameters 
    validation upto some maximum split point and yields total
    return as output (negative for hyperopt purposes)
    
    Parameters:
    -------
    params - dictionary with hyperparameters from hyperopt
                and 'upto' key, that limits cross-validation in time,
                and 'tick' key, which identifies the ticker for which
                        the hyperparameters are optimized
                
    Returns:
    --------
    average matt score for the cross-validation
    """
    
    params = integerize(params)
    results = []
    
    my_model = LGBMClassifier(num_leaves=params['num_leaves'],
                                         max_depth=params['max_depth'],
                                         learning_rate=params['learning_rate'],
                                         n_estimators=params['n_estimators'],
                                         min_child_samples=params['min_child_samples'])
    
    for split in range(params['upto']-500, params['upto'], 100):
        
        trainx, testx, trainy, testy = my_tss(tickers_dfs[params['tick']], split, 
                                                  train_size=params['train_size'], 
                                                  test_size=100)

        sw = np.where(trainy==0, params['sw'], 1)

        my_model.fit(trainx, trainy, sample_weight=sw)
        
        testx['pred'] = my_model.predict(testx)
        testx['fwd'] = tickers_dfs[tick]['fwd']
        testx['avg'] = np.where(testx['pred']==1, 1+testx['fwd'], 1)
        results.append(testx)
       
    return calc_ret(results)


In [ ]:
"""
Calculating return for Model F20.
First 20 stocks only.
"""

modelF20_returns = []

for upto in tqdm_notebook(range(1500, 2100, 100)):
    
    tick_params = {}
    
    for tick in df.columns[:20]:
        
        def bayes_opt_modelF(tick, upto, eval_n=100):
            space_index = {
                'num_leaves': hp.quniform('num_leaves', 10, 50, 5),
                'max_depth': hp.quniform('max_depth', 3, 8, 1),
                'learning_rate': hp.loguniform('learning_rate', np.log(0.05), np.log(0.2)),
                'n_estimators': hp.quniform('n_estimators', 32, 256, 8),
                'min_child_samples': hp.quniform('min_child_samples', 10, 50, 5),
                'sw': hp.uniform('sw', 0.4, 0.7),
                'train_size': hp.quniform('train_size', 200, 1000, 100),
                'upto': hp.choice('upto', [upto]),
                'tick': hp.choice('tick', [tick])
            }
            tpe_algo = tpe.suggest
            tpe_trials = Trials()
            tpe_best = fmin(fn=analyze_upto_modelF, space=space_index, algo=tpe_algo, trials=tpe_trials, 
                            max_evals=eval_n)
            return tpe_best
        
        tick_params[tick] = bayes_opt_modelF(tick, upto, eval_n=100)
    
    modelF20_returns.append(check_paramsEF(tick_params, upto))

modelF20_ret = get_est_series(pd.concat(modelF20_returns))

print("Average return for Model F20:", \
      round(modelF20_ret.iloc[-1]-100, 2), '%')